In [1]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays 

# S(arg)

it generate (and return)a scaling matrix. the returned matrix have d+1 row and column where d is the lenght of the array passed as arg

In [3]:
function s(args...)
	d = length(args)
	mat = Matrix{Float64}(LinearAlgebra.I, d+1, d+1)
	for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

In [4]:
@btime s(-0.5,-0.5)

  44.141 ns (1 allocation: 160 bytes)


3×3 Matrix{Float64}:
 -0.5   0.0  0.0
  0.0  -0.5  0.0
  0.0   0.0  1.0

In [4]:
@code_llvm s(-0.5,-0.5)

;  @ In[2]:1 within `s'
; Function Attrs: uwtable
define nonnull {}* @julia_s_1803(double %0, double %1) #0 {
pass:
;  @ In[2]:3 within `s'
; ┌ @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:471 within `Array'
   %2 = call nonnull {}* @j_Array_1805([1 x i8]* nocapture readonly @_j_const1, [2 x i64]* nocapture readonly @_j_const2) #0
; └
;  @ In[2]:5 within `s'
; ┌ @ array.jl within `setindex!'
   %3 = bitcast {}* %2 to {}**
   %4 = bitcast {}* %2 to double**
   %5 = getelementptr inbounds {}*, {}** %3, i64 3
   %6 = bitcast {}** %5 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %7 = load i64, i64* %6, align 8
   %.not = icmp eq i64 %7, 0
   br i1 %.not, label %oob, label %ib

ib:                                               ; preds = %pass
; └
; ┌ @ array.jl within `setindex!'
   %8 = getelementptr inbounds {}*, {}** %3, i64 4
   %9 = bitcast {}** %8 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %10 = load i64, i64

In [5]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  47.411 ns …  1.637 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     53.807 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   68.344 ns ± 56.678 ns  ┊ GC (mean ± σ):  2.72% ± 5.04%

  ▇█▅▆▄▃▄▁▁▂                   ▁                              ▂
  ████████████▇▇▇▆▇▇▇█▇▇▇▇▆▆▅▆██████▇▆▇▇▇▆▆▅▄▆▆▅▅▁▁▅▄▃▄▄▁▄▃▄▄ █
  47.4 ns      Histogram: log(frequency) by time       297 ns <

 Memory estimate: 160 bytes, allocs estimate: 1.

# Improvment & changes

In [15]:
function s(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @inbounds for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

In [16]:
@btime s(-0.5,-0.5)

  11.400 ns (1 allocation: 80 bytes)


3×3 MMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.5   0.0  0.0
  0.0  -0.5  0.0
  0.0   0.0  1.0

In [17]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  11.512 ns …  2.270 μs  ┊ GC (min … max):  0.00% … 97.65%
 Time  (median):     12.312 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.113 ns ± 83.803 ns  ┊ GC (mean ± σ):  18.75% ±  4.04%

  ██▇▄▃▂▁▁ ▁▂▃▂▁                                    ▁ ▂▁      ▂
  ███████████████▇▇▆▆▆▆▆▇▆▇▅▅▅▅▆▅▆▄▆▅▄▄▆▄▄▅▄▄▃▃▃▁▃▇▅████▇▇▆▅▅ █
  11.5 ns      Histogram: log(frequency) by time        52 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# Paralelization

In [4]:
function s(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @async for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

## sync and async made the funcion slower

In [5]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 8471 samples with 267 evaluations.
 Range (min … max):  471.536 ns …  4.760 ms  ┊ GC (min … max):  0.00% … 99.96%
 Time  (median):     974.532 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.171 μs ± 59.395 μs  ┊ GC (mean ± σ):  48.82% ±  2.17%

             ▁█▅                                                
  ▁▂▁▂▂▂▂▂▂▂▄████▆▄▄▃▂▂▂▂▂▂▂▂▂▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  472 ns          Histogram: frequency by time         2.64 μs <

 Memory estimate: 608 bytes, allocs estimate: 6.

In [6]:
function s(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @simd for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

## With the usage of simd we improve the old funcion but we still got worse timing

In [5]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  15.331 ns …  15.582 μs  ┊ GC (min … max):  0.00% … 99.06%
 Time  (median):     26.653 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   53.992 ns ± 381.735 ns  ┊ GC (mean ± σ):  27.54% ±  4.07%

  ▄▃▅▃▆█▃▂▂▂▁▂▂▆▅▂▃▃▁▂▂▁▁   ▂▂▁  ▁▁                            ▂
  ██████████████████████████████████▇██▇▇▆▇▇▆▇▆▆▇▅▅▆▅▆▅▆▅▅▆▄▄▅ █
  15.3 ns       Histogram: log(frequency) by time       136 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# Using threads the funcion is faster then the async but slowen then simd
## (tested with 4 threads)

In [7]:
function s(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    Threads.@spawn for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

In [8]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 310 evaluations.
 Range (min … max):  402.258 ns … 28.082 μs  ┊ GC (min … max): 0.00% … 95.78%
 Time  (median):     912.903 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.021 μs ±  1.496 μs  ┊ GC (mean ± σ):  9.22% ±  6.07%

                      ▂▅▇█▅▄▁                                   
  ▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▅▆███████▆▄▃▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  402 ns          Histogram: frequency by time         1.72 μs <

 Memory estimate: 616 bytes, allocs estimate: 6.